In [1]:
from pyspark import SparkConf,SparkContext
from operator import add
from tqdm import tqdm_notebook as tqdm
from fractions import *
from math import exp
from pyspark import StorageLevel
from tempfile import NamedTemporaryFile

In [2]:
StorageLevel.MEMORY_AND_DISK

StorageLevel(True, True, False, False, 1)

# Todo
[ ] 修正權重  
[ ] Personal Page rank  

In [3]:
conf = SparkConf().setMaster('local').setAppName('Test_APP_2')
conf.set("spark.executor.cores", "1")
# conf = SparkConf().setMaster("local[*]").setAppName('MatrixMultiplication')

conf.set('spark.executor.memory', '24G') \
    .set('spark.driver.memory', '30G') \
    .set('spark.driver.maxResultSize', '20G')
sc = SparkContext(conf=conf)

In [4]:
# file = 'dev.txt'
# file = 'HatePolitics_u2u_interactive_graph/part-00000'
file = 'u2u_interactive_graph/part-00000'
beta = 0.8
error_range = 10**-10

In [5]:
# 讀入黑名單
# with open('data/Bad_id.txt') as f:
with open('new_bad_id') as f:
    
    BAD_ID = f.readlines()
    BAD_ID = [i.strip() for i in BAD_ID]
    print(BAD_ID[:10])
    BAD_ID = set(BAD_ID)
    BAD_ID_Number =len(BAD_ID)


['CLV518', 'SUPERBOWL', 'nook777', 'bouly', 'qqee', 'tonyselina', 'mattlane', 'Aptantion', 'goodjeff', 'hotelss']


# 資料讀取跟前處理
將資料讀入，並轉成 (key,[value, this_node_connect_to_which_node]) 的格式

In [6]:
%%time
# 單純的資料讀取跟前處理
def line_to_stochastic_matrix(line):
    return_value = eval(line)
#     source, dest = list(map(int, line.split('\t')))
#     return_value.append([source, -1])
#     return_value.append([dest, -1])
#     return_value.append([source, dest])
    return return_value


    
stochastic_matrix = sc.textFile(file).map(line_to_stochastic_matrix)
stochastic_matrix = stochastic_matrix.groupByKey().mapValues(sum)
N = stochastic_matrix.flatMap(lambda x:x[0]).distinct().count()

stochastic_matrix = stochastic_matrix.map(lambda x:(x[0][0], (x[0][1], x[1]))).groupByKey() # (source,[(dist,weight)])

for i in stochastic_matrix.take(5):
    print(i[0], list(i[1]))

mihono [('timmy1217', 1), ('abiese05', 1), ('ching1210', 1), ('richshen', 6), ('konny', 1), ('mnhyuiop', 5), ('KobeRapist', 1), ('HomeTogether', 1), ('mmm851010', 1), ('twyes', 1), ('silvertime', 1), ('GMQQQ', 1), ('winddin', 1), ('cattgirl', 1), ('xxxluke', 1), ('derekgao', 1), ('SilverFlare', 2), ('LoveMakeLove', 3), ('a12141623', 1), ('diefishfish', 2), ('mrgod', 1), ('qazStarStar', 2), ('lunli924', 6), ('GodBull', 1), ('HCS0730', 1), ('number89757', 1), ('paericsson', 7), ('lovelypag', 1), ('momoispig', 1), ('krit1009', 1), ('bear26', 1), ('Punchline', 1), ('tung3567752', 1), ('bomda', 1), ('A1bertPujols', 2), ('DarkHolbach', 1), ('KMT5566', 1), ('kazuyaya', 1), ('cake10414', 1), ('ClownT', 2), ('Sinreigensou', 4), ('mikejason38', 1), ('THINKTAIWAN', 1), ('trashpoking', 1), ('whour', 1), ('ilovesongyy', 1), ('Rikako4650', 1), ('hvariables', 2), ('MoriiKaho', 2), ('Kaoos', 3), ('XXXXOAY', 1), ('kmtrash', 1), ('dandandanbao', 2), ('liveforhappy', 5), ('LEDtorch', 1), ('oomen', 1), ('

In [7]:
%%time
# sum([])
# BAD_ID = stochastic_matrix.map(lambda x:x[0]).collect()[:100]
# BAD_ID = set(BAD_ID)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.34 µs


In [8]:
%%time
def turn_to_stochastic_matrix(item, N, ban_id):
    source, dests = item
    dests = [[i[0], exp(-1 * ( (i[1]**2) / (2 * beta))) ] for i in dests]
    goto = []
    total_weight = sum([i[1] for i in dests])
    for i in dests:
        dest, weight = i
        if total_weight == 0:
            weight = 0
        else:
            weight = weight/total_weight
        

        goto.append([dest, weight])
    
#     return key, [Fraction(1, N), goto]
    if source in ban_id:
        return (source, [1, goto])
    else:
        return (source, [0, goto])
    
stochastic_matrix = stochastic_matrix.map(lambda x: turn_to_stochastic_matrix(x, N, BAD_ID))
for i in stochastic_matrix.take(5):
    print(i[0], list(i[1]))

mihono [0, [['timmy1217', 0.0007754237338030624], ['abiese05', 0.0007754237338030624], ['ching1210', 0.0007754237338030624], ['richshen', 2.451022499412777e-13], ['konny', 0.0007754237338030624], ['mnhyuiop', 2.372039195425468e-10], ['KobeRapist', 0.0007754237338030624], ['HomeTogether', 0.0007754237338030624], ['mmm851010', 0.0007754237338030624], ['twyes', 0.0007754237338030624], ['silvertime', 0.0007754237338030624], ['GMQQQ', 0.0007754237338030624], ['winddin', 0.0007754237338030624], ['cattgirl', 0.0007754237338030624], ['xxxluke', 0.0007754237338030624], ['derekgao', 0.0007754237338030624], ['SilverFlare', 0.00011891508098813926], ['LoveMakeLove', 5.224764020197991e-06], ['a12141623', 0.0007754237338030624], ['diefishfish', 0.00011891508098813926], ['mrgod', 0.0007754237338030624], ['qazStarStar', 0.00011891508098813926], ['lunli924', 2.451022499412777e-13], ['GodBull', 0.0007754237338030624], ['HCS0730', 0.0007754237338030624], ['number89757', 0.0007754237338030624], ['paericsso

In [9]:
BAD_ID_Number

1371

In [10]:
%%time
tmp2 = stochastic_matrix.filter(lambda x:x[0] in BAD_ID).count()
tmp2

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 34 s


1344

In [11]:
# %%time
# tmp = stochastic_matrix.map(lambda x:x[1][0]).sum()
# tmp

In [12]:
# tmp2 = stochastic_matrix.map(lambda x:x[1]).collect()

In [13]:
# len(tmp2)


In [14]:
# tmp2 = set(tmp2)
# len(tmp2)

In [15]:
# BAD_ID_Number

In [16]:
# BAD_ID & tmp2

In [17]:
# %%time
# stochastic_matrix = stochastic_matrix.map(lambda x: turn_to_stochastic_matrix(x, N, BAD_ID))
# # [(i, list(j)) for i,j in stochastic_matrix.collect()]
# stochastic_matrix.take(5)

In [18]:
# 格式是
# source [(dist,(weight, vale))]
# for i in stochastic_matrix.take(5):
#     print(i[0], list(i[1]))

# Mapper
把所有 node 對其他 node 的貢獻都算出來

In [19]:
%%time
def get_new_value(item, beta, N, bad_id):
    source, v = item
    value, dests = v
#     weight, value = value
#     source, dest = key
#     total_weight = len([i[1][0] for i in dests])
    
    new_values = []
    for i in dests:
        dest, weight = i
#         new_values.append([i, beta * (value/ N_dest) + (1-beta)/N])
        # 跳到 bad_id (personal page rank)
#         if dest in bad_id: #
#             new_values.append([i[0], beta * (value * weight) +  ((1-beta)/len(bad_id))])
#         else:
#             new_values.append([i[0], beta * (value * weight)] )
        new_values.append([i[0], beta * (value * weight)] )
        
    return new_values

new_values = stochastic_matrix.flatMap(lambda x: get_new_value(x, beta, N, BAD_ID))
new_values.take(30)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 923 ms


[['timmy1217', 0.0],
 ['abiese05', 0.0],
 ['ching1210', 0.0],
 ['richshen', 0.0],
 ['konny', 0.0],
 ['mnhyuiop', 0.0],
 ['KobeRapist', 0.0],
 ['HomeTogether', 0.0],
 ['mmm851010', 0.0],
 ['twyes', 0.0],
 ['silvertime', 0.0],
 ['GMQQQ', 0.0],
 ['winddin', 0.0],
 ['cattgirl', 0.0],
 ['xxxluke', 0.0],
 ['derekgao', 0.0],
 ['SilverFlare', 0.0],
 ['LoveMakeLove', 0.0],
 ['a12141623', 0.0],
 ['diefishfish', 0.0],
 ['mrgod', 0.0],
 ['qazStarStar', 0.0],
 ['lunli924', 0.0],
 ['GodBull', 0.0],
 ['HCS0730', 0.0],
 ['number89757', 0.0],
 ['paericsson', 0.0],
 ['lovelypag', 0.0],
 ['momoispig', 0.0],
 ['krit1009', 0.0]]

# Reducer
加總所有貢獻

In [20]:
%%time
def topic_rank(x, bad_id):
    if x[0] in bad_id:
        return [x[0], beta * (x[1] + ((1-beta)/len(bad_id))) + ((1-beta) * 1)]
    else:
        return [x[0], beta * x[1]]
new_values = new_values.reduceByKey(add).map(lambda x: topic_rank(x, BAD_ID) )
new_values.take(5)

CPU times: user 32 ms, sys: 4 ms, total: 36 ms
Wall time: 52.3 s


[['abiese05', 0.015990393285389728],
 ['ching1210', 0.08966110883005907],
 ['mmm851010', 0.2061291253489559],
 ['cattgirl', 0.026601978036987974],
 ['a12141623', 0.003018249861259281]]

# 算出 S

In [21]:
# S = new_values.values().reduce(add)
# S

In [22]:
%%time
S = new_values.values().sum()
S

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 763 ms


939.2060977388742

# 結合 new 跟 old 為之後的更新做準備

In [23]:
# def reducer(x):

new_stochastic_matrix = stochastic_matrix.fullOuterJoin(new_values)
new_stochastic_matrix.take(1)

# [(x, list(map(list, y))) for x, y in sorted(list(new_stochastic_matrix.collect()))]


[('fluterway',
  ([0,
    [['hongou', 0.0010812444683052599],
     ['HateAnus', 0.0010812444683052599],
     ['qope', 0.0010812444683052599],
     ['Ho1liday', 7.285367920495186e-06],
     ['ccugoo', 0.0010812444683052599],
     ['billy3321', 0.0010812444683052599],
     ['Liaslu', 0.0010812444683052599],
     ['JPGstar', 0.0010812444683052599],
     ['zxcxxx', 0.0010812444683052599],
     ['mexxbaby', 0.0010812444683052599],
     ['venson', 0.00016581420958821543],
     ['waymayday', 0.0010812444683052599],
     ['linwahaha', 0.0010812444683052599],
     ['caeasonfb', 0.0010812444683052599],
     ['chadcooper', 0.0010812444683052599],
     ['ForTheHorde', 0.0010812444683052599],
     ['viodin', 0.0010812444683052599],
     ['TBOC', 0.0010812444683052599],
     ['ytkuang', 0.0010812444683052599],
     ['robert167', 0.0010812444683052599],
     ['ios6', 0.0010812444683052599],
     ['semih', 0.00016581420958821543],
     ['sss1234', 0.0010812444683052599],
     ['Caress', 0.001081244468

In [24]:
new_stochastic_matrix.take(1)[0][1][1]

0.0

In [25]:
N

288408

# 更新舊的數值

In [26]:
def sum_new_value(item, S, N, estimate_S):
    key, value = item
    
    if value[1] == None :
        new_value = 0 + (estimate_S-S)/N
    else:
        new_value = value[1] + (estimate_S-S)/N

    
    if value[0] == None:
        return [key, [new_value, []]]
    else:
        return [key, [new_value, value[0][1]]]

estimate_S = len(BAD_ID)
new_stochastic_matrix = new_stochastic_matrix.map(lambda x: sum_new_value(x, S, N, estimate_S))

# sorted(stochastic_matrix.collect())[:10]
new_stochastic_matrix.take(10)

[['fluterway',
  [0.001497163401365863,
   [['hongou', 0.0010812444683052599],
    ['HateAnus', 0.0010812444683052599],
    ['qope', 0.0010812444683052599],
    ['Ho1liday', 7.285367920495186e-06],
    ['ccugoo', 0.0010812444683052599],
    ['billy3321', 0.0010812444683052599],
    ['Liaslu', 0.0010812444683052599],
    ['JPGstar', 0.0010812444683052599],
    ['zxcxxx', 0.0010812444683052599],
    ['mexxbaby', 0.0010812444683052599],
    ['venson', 0.00016581420958821543],
    ['waymayday', 0.0010812444683052599],
    ['linwahaha', 0.0010812444683052599],
    ['caeasonfb', 0.0010812444683052599],
    ['chadcooper', 0.0010812444683052599],
    ['ForTheHorde', 0.0010812444683052599],
    ['viodin', 0.0010812444683052599],
    ['TBOC', 0.0010812444683052599],
    ['ytkuang', 0.0010812444683052599],
    ['robert167', 0.0010812444683052599],
    ['ios6', 0.0010812444683052599],
    ['semih', 0.00016581420958821543],
    ['sss1234', 0.0010812444683052599],
    ['Caress', 0.001081244468305259

In [27]:
S = new_stochastic_matrix.values().map(lambda x:x[0]).reduce(add)
S

1371.000000000399

In [28]:
%%time
def check_end(x, error_range):

    old, new = x[1]
    if old == None or new == None:
        return False
    return abs(old - new) < error_range

V_new_stochastic_matrix = new_stochastic_matrix.map(lambda x: (x[0],x[1][0]))
V_stochastic_matrix = stochastic_matrix.map(lambda x:(x[0],x[1][0]))
is_end = V_stochastic_matrix.fullOuterJoin(V_new_stochastic_matrix).map(lambda x:check_end(x, error_range)).sum() == V_new_stochastic_matrix.count()
# new_stochastic_matrix
# stochastic_matrix

CPU times: user 52 ms, sys: 4 ms, total: 56 ms
Wall time: 1min 11s


In [29]:
is_end


False

In [30]:
stochastic_matrix = new_stochastic_matrix

In [31]:
# stochastic_matrix.saveAsSequenceFile('tttttt')

In [32]:
# %%time
# tmpFile = NamedTemporaryFile(delete=True)
# tmpFile.close()
# stochastic_matrix = new_stochastic_matrix
# stochastic_matrix.saveAsTextFile(tmpFile.name)
# stochastic_matrix = sc.wholeTextFiles(tmpFile.name)

In [33]:
# new_stochastic_matrix.getNumPartitions()

In [34]:
# t = stochastic_matrix.map(lambda x:x[1])

In [35]:
# def fileToRDD(x):
#     x.strip()
#     x = x.split('\n')
#     x = list(filter(lambda x: x.strip() != '', x))
#     x = list(map(eval,x))
#     return x
# stochastic_matrix = stochastic_matrix.repartition(1).values().flatMap(fileToRDD)
# t = stochastic_matrix.repartition(1).values().flatMap(fileToRDD)

In [36]:
# t.getNumPartitions()

In [37]:
# t.take(1)

In [38]:
# t = sc.parallelize(t)

In [39]:
# new_stochastic_matrix.take(1)

In [40]:
# t.take(1)

In [41]:
# %%time
# stochastic_matrix = new_stochastic_matrix
# stochastic_matrix = stochastic_matrix.collect()
# stochastic_matrix = sc.parallelize(stochastic_matrix)

In [42]:
# %%time
# new_values = stochastic_matrix.flatMap(lambda x: get_new_value(x, beta, N, BAD_ID))

In [43]:
# %%time
# new_values = new_values.reduceByKey(add).map(lambda x: topic_rank(x, BAD_ID) )

In [44]:
# %%time
# S = new_values.values().sum()

# 上面那些流程多跑幾次，直到收斂

In [45]:
%%time
from time import time

# for idx, i in tqdm(enumerate(range(19))):
idx = 0
while not is_end:
    
    start_time = time() # <-- start timing
    
    new_values = stochastic_matrix.flatMap(lambda x: get_new_value(x, beta, N, BAD_ID))
#     print(f'{idx} iter')
#     new_values.cache()
    new_values = new_values.reduceByKey(add).map(lambda x: topic_rank(x, BAD_ID) )
    
#     print(f'{idx} check problem {new_values.count()}')
    S = new_values.values().sum()
    
#     print(f'{idx} check problem')
    new_stochastic_matrix = stochastic_matrix.fullOuterJoin(new_values)
    
#     S = new_values.values().sum()
#     print(f'{idx} check problem')
#     new_stochastic_matrix = new_stochastic_matrix.map(lambda x: sum_new_value(x, S, N))
#     estimate_S = stochastic_matrix.count()
    estimate_S = len(BAD_ID)
    new_stochastic_matrix = new_stochastic_matrix.map(lambda x: sum_new_value(x, S, N, estimate_S))

    
#     print(f'{idx} check problem')
    V_new_stochastic_matrix = new_stochastic_matrix.map(lambda x: (x[0],x[1][0]))
    
#     print(f'{idx} check problem')
    V_stochastic_matrix = stochastic_matrix.map(lambda x:(x[0],x[1][0]))
    
#     print(f'{idx} check problem')
    N_True = V_stochastic_matrix.fullOuterJoin(V_new_stochastic_matrix).map(lambda x:check_end(x, error_range)).sum()
    N_new = V_new_stochastic_matrix.count()
    is_end = (N_True == N_new)
#     print(f'{idx} iter end')
    
#     tmpFile = NamedTemporaryFile(delete=True)
#     tmpFile.close()
    stochastic_matrix = new_stochastic_matrix
    stochastic_matrix = stochastic_matrix.repartition(1) 
#     stochastic_matrix.saveAsTextFile(tmpFile.name)
#     stochastic_matrix = sc.wholeTextFiles(tmpFile.name)
#     stochastic_matrix = stochastic_matrix.repartition(1).values().flatMap(fileToRDD)

#     stochastic_matrix = stochastic_matrix.cache()
#     stochastic_matrix = stochastic_matrix.collect()
#     stochastic_matrix = sc.parallelize(stochastic_matrix)

        
    end_time = time()
    print(idx, end_time - start_time, N_True, N_new, new_values.count(), S)
    idx += 1

    

0 119.2936270236969 245945 288408 45281 645.8916324839648
1 196.96663117408752 0 288408 45281 929.9714257302504
2 230.8820357322693 0 288408 45281 921.8141557087163
3 231.92906427383423 0 288408 45281 922.7623180342977
4 231.94000458717346 2 288408 45281 922.8348680599081
5 230.27453899383545 4 288408 45281 922.8509783142828
6 232.69895720481873 9 288408 45281 922.8696919150112
7 230.36380290985107 4 288408 45281 922.8756111662491
8 230.7685477733612 1 288408 45281 922.8787232877365
9 231.067138671875 0 288408 45281 922.8804065927692
10 233.90676069259644 0 288408 45281 922.8814442660855
11 237.94948506355286 0 288408 45281 922.8820841294853
12 236.60476160049438 0 288408 45281 922.88249289924
13 237.8161976337433 0 288408 45281 922.8827528194001
14 231.84801316261292 0 288408 45281 922.8829198497667
15 232.50149774551392 0 288408 45281 922.8830265419949
16 231.27983379364014 0 288408 45281 922.8830950135506
17 231.17268681526184 0 288408 45281 922.883138781145
18 231.879780292511 2438

In [46]:
stochastic_matrix.saveAsTextFile('real_result_estimate-l-en-bad_test')

# 輸出結果

In [47]:
output = sorted(stochastic_matrix.collect(), key=lambda x: x[1][0], reverse=True)

with open('output_estimate-len-bad_test.txt', 'w') as f:
    for i in output:
        f.write(f'{i[0]}\t{i[1][0]}\n')

In [48]:
with open('output_estimate-len-bad_test.txt') as f:
    o = f.readlines()
    
for i in range(10):
    print(o[i].strip())

seabox	3.2885363203816476
iPad3	3.0303901517819996
Bignana	2.9753880782441335
XXXXGAY	2.8203715052438296
Kay731	2.2689163661266707
mayaman	2.092478878861052
okcool	1.9230319217233514
gn01765288	1.455196891111766
hateOnas	1.4348554763935482
maxmaster	1.3742075998215988


In [49]:
# !pip install networkx
# stochastic_matrix.count()

In [50]:
# import networkx as nx
# G=nx.DiGraph()


In [51]:
# with open(file) as f:
#     edge = f.readlines()
#     edge = [i.strip().split('\t') for i in edge]
    
# G.add_edges_from(edge)
    

In [52]:
# for i in edge:
#     G.add_edge(*i)

In [53]:
# pr = nx.pagerank(G, alpha=0.8)
# pr = sorted(list(pr.items()), key=lambda x:x[1], reverse=True)

In [54]:
# pr[:20]

In [55]:

# from fractions import *
# a = Fraction(10,8)
# a/2

In [56]:
# !head -n 10 output_estimate-len-bad_test.txt

In [57]:
# !tail -n 10 output_estimate-len-bad_test.txt